# Atari

In [37]:
import numpy as np
import tensorflow as tf
from buffer import Buffer
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Lambda, concatenate
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Flatten
from keras.optimizers import Adam 
from keras.optimizers import RMSprop
from keras import backend as K
from keras.utils import to_categorical

import gym
env = gym.make('Breakout-v0')
env.observation_space
output_dim = env.action_space.n

def to_grayscale(img):
    return np.mean(img, axis=2).astype(np.uint8)

def downsample(img):
    return img[::2, ::2]

def preprocess(img):
    return to_grayscale(downsample(img))

def transform_reward(reward):
    return np.sign(reward)


frame = env.reset()
n_actions = output_dim

# We assume a theano backend here, so the "channels" are first.
ATARI_SHAPE = (105, 80, 4)

# With the functional API we need to define the inputs.
frames_input = Input(shape = ATARI_SHAPE, name='frames')
actions_input = Input((n_actions,), name='mask')

# Assuming that the input frames are still encoded from 0 to 255. Transforming to [0, 1].
normalized = Lambda(lambda x: x / 255.0)(frames_input)

conv_1 = Convolution2D(32, (3, 3), activation='relu', data_format='channels_first')(normalized)
conv_2 = Convolution2D(32, (3, 3), activation='relu')(conv_1)
# Flattening the second convolutional layer.
conv_flattened = Flatten()(conv_2)
# "The final hidden layer is fully-connected and consists of 256 rectifier units."
hidden = Dense(256, activation='relu')(conv_flattened)
# "The output layer is a fully-connected linear layer with a single output for each valid action."
output = Dense(n_actions)(hidden)
# Finally, we multiply the output by the mask!
#filtered_output = merge([output, actions_input], mode='mul')
filtered_output = concatenate([output, actions_input])
model = Model(input=[frames_input, actions_input], output=filtered_output)
optimizer = RMSprop(lr=0.00025, rho=0.95, epsilon=0.01)
model.compile(optimizer, loss='mse')

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:69: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("co..., inputs=[<tf.Tenso...)`


In [94]:
#inp = Input(shape=state.shape)
batch_size, width, height = 10, 3, 4
inp = Input(shape=(width,height,batch_size))
x = Dense(4,activation='relu')(inp)
model = Model(input=inp,output=x)

#frame = env.reset()
#frame = preprocess(frame)
#state = np.zeros((frame.shape[0],frame.shape[1],4))
state = np.zeros((3,4,batch_size))
out = model.predict(state)

/home/kokeeffe/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """


ValueError: Error when checking input: expected input_41 to have 4 dimensions, but got array with shape (3, 4, 10)

In [92]:
frame.shape

(105, 80)

### Random